# OpenStreetMap Project - Tampa, Fl

### Map Area

Tampa, FL, United States

* https://www.openstreetmap.org/relation/1216849
* https://mapzen.com/data/metro-extracts/metro/tampa_florida/
(417.5MB)

Tampa is the city I live in and familiar with, This project is to apply the techniques learned from Udacity's Data Wrangling course to parse, audit and clean this dataset then convert xml into csv, and using sql queries to explore some facts about this city.

I experimented with a smaller .osm datafile named temple_terrace.osm(10.4MB), which represents my neighborhood area.

## Data Auditing and Problems Encountered

* Examined element types and occurrence, this part looks fine. **(data_overview.py)**
>Element types and occurrence of temple_terrace.osm<br />
>[('nd', 2195579),<br />
> ('node', 1822389),<br />
> ('tag', 916600),<br />
> ('way', 291596),<br />
> ('member', 17721),<br />
> ('relation', 670),<br />
> ('osm', 1),<br />
> ('bounds', 1)]<br />


* Examined type of keys, most of them were good, 3 keys contain problem characters, which are actually blank space.
>{'lower': 710176, 'lower_colon': 194348, 'other': 12073, 'problemchars': 3}<br />
>service area<br />
>Payments Accepted<br />
>Payments Accepted<br />

* Examined occurrence of keys, very few of them used deprecated notation, suffixed name tagging, such as 'cycleway_1', 'natural_1'.
* Examined values for postcode, for the part of map I chose, postcodes are all numbers, most of them used standard five digits postcode, but a few used nine digits 'XXXXX-XXXX'. I will drop the '-XXXX' part. **(audit_clean_postcode.py)**
* Examined values for street types, many of them are abbreiviated in different ways, such as 'Dr.', 'dr', 'Dr'for 'Drive', and 'Ave.','av','Ave'for 'Avenue'. There are also some cases of misspelling, such as 'Bolevard'. I will fix those and use standard notations.**(audit_clean_street.py)**


## Data cleaning and database import

### clean up postcode
A count_postcodes() function was created to audit the postcodes in the area, and the output shows that the only issue of format inconsistency is the four digits suffix.

In [23]:
def count_postcodes(filename):
    postcodes = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key == 'addr:postcode':
                postcode = elem.attrib.get('v')
                if postcode not in postcodes:
                    postcodes[postcode] = 1
                else:
                    postcodes[postcode] += 1
    return postcodes

postcodes = count_postcodes(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in postcodes.items()], reverse=True)]

print 'Postcode values and occurrence in tampa.osm'
pprint.pprint(sorted_by_occurrence)


Postcode values and occurrence in tampa.osm
[('33612', 167),
 ('33702', 120),
 ('33511', 109),
 ('33607', 92),
 ('33602', 89),
 ('33703', 88),
 ('33704', 79),
 ('33781', 74),
 ('33606', 71),
 ('33714', 67),
 ('33617', 67),
 ('33624', 64),
 ('33716', 59),
 ('33635', 59),
 ('33578', 57),
 ('33626', 49),
 ('34677', 45),
 ('33614', 41),
 ('33619', 37),
 ('33634', 34),
 ('33609', 32),
 ('33569', 30),
 ('33527', 27),
 ('33604', 26),
 ('33605', 24),
 ('33629', 23),
 ('33603', 22),
 ('33615', 21),
 ('33611', 21),
 ('33610', 21),
 ('33556', 20),
 ('33613', 19),
 ('33713', 18),
 ('33594', 18),
 ('33592', 18),
 ('33618', 16),
 ('33762', 14),
 ('33547', 14),
 ('33729', 13),
 ('33710', 12),
 ('33596', 8),
 ('33647', 7),
 ('33544', 7),
 ('33572', 6),
 ('33558', 6),
 ('33534', 6),
 ('34655', 5),
 ('33637', 5),
 ('33584', 5),
 ('33559', 5),
 ('34639', 4),
 ('33549', 4),
 ('33625', 3),
 ('33620', 3),
 ('33616', 3),
 ('34695', 2),
 ('34677-6300', 2),
 ('33643', 2),
 ('33579', 2),
 ('33548', 2),
 ('33510

A clean_postcode() was created to fix the inconsistency, it simply takes the first five digits of postcode_value.

In [24]:
def clean_postcode(postcode_value):

    if len(postcode_value)!=5:
        postcode=postcode_value[0:5]
                        
        return postcode

### standardize abbreviated street names<br />

In order to standardize abbreviated street names, in audit_clean_street.py, update_name() function updates street name based on the manually created mapping dictionary.

>Penrod Ln => Penrod Lane<br />
Whitmarsh Ln => Whitmarsh Lane<br />
Webb Rd => Webb Road<br />
Hutchison Rd => Hutchison Road<br />
Race Track Rd => Race Track Road<br />
Kelly Rd => Kelly Road<br />
Providence Rd => Providence Road<br />
Anderson Rd => Anderson Road<br />
Orient Rd => Orient Road<br />
Tampa Rd => Tampa Road<br />
Ulmerton Rd => Ulmerton Road<br />
George Rd => George Road<br />
Tarpon Springs Rd => Tarpon Springs Road<br />
Ehrlich Rd => Ehrlich Road<br />
Sheldon Rd => Sheldon Road<br />
Palm Pointe Dr. => Palm Pointe Drive<br />
9201 W. Waters Ave. => 9201 W. Waters Avenue<br />



In [25]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type_re, mapping[street_type], name)

    return name

### file conversion and database import

* shape_element() from to_csv.py parses the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files. clean_postcode() and update_name() are also plugged in so that while iterating through the osm file, the data get cleaned up.**(to_csv.py)**
* tampa.db was created and five tables('node', 'node_tags','way','way_nodes','way_tags') were created using the designated schemas, corresponding csv data were imported to tables. 

## Data exploration

### file size

In [58]:
import os
print('The tampa_cleaned.xml file is {} MB'.format(os.path.getsize('tampa_cleaned.xml')/1.0e6))
print('The tampa.db file is {} MB'.format(os.path.getsize('tampa.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

The tampa_cleaned.xml file is 422.40293 MB
The tampa.db file is 147.521536 MB
The nodes.csv file is 91.539394 MB
The nodes_tags.csv file is 91.539394 MB
The ways.csv file is 19.066574 MB
The ways_tags.csv is 27.487402 MB
The ways_nodes.csv is 24.777723 MB


### number of nodes

In [59]:
import sqlite3
sqlite_file = "tampa.db"
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())

[(1048575,)]


### number of ways

In [60]:
cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(291596,)]


### number of unique users

In [61]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print(cur.fetchall())

[(893,)]


### Top 10 contributing users

In [75]:
import pprint
cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())

[(u'Andrew Matheny_import', 404260),
 (u'ninja_import', 94536),
 (u'EdHillsman', 91743),
 (u'woodpeck_fixbot', 59404),
 (u'coleman', 58410),
 (u'David Hey', 57929),
 (u'KalininOV', 57366),
 (u'grouper', 39270),
 (u'jharpster', 37479),
 (u'Jack the Ripper', 32749)]


### tourism related categories descending

In [63]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%tourism'\
             GROUP BY tags.value \
             ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[(u'hotel', 115),
 (u'information', 57),
 (u'motel', 50),
 (u'attraction', 21),
 (u'viewpoint', 16),
 (u'museum', 15),
 (u'artwork', 12),
 (u'picnic_site', 12),
 (u'theme_park', 9),
 (u'camp_site', 6),
 (u'zoo', 4),
 (u'caravan_site', 3),
 (u'terminal', 2),
 (u'apartment', 1),
 (u'aquarium', 1),
 (u'guest_house', 1),
 (u'hostel', 1),
 (u'zoo;attraction', 1)]


### top 5 cafe chain

In [68]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='cafe') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u'Starbucks', 15),
 (u"Dunkin' Donuts", 3),
 (u'Starbucks Coffee', 3),
 (u'Kaleisia Tea Lounge', 2),
 (u'Tropical Smoothie Cafe', 2)]


### top 5 popular fast food chain

In [64]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='fast_food') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u'Subway', 29),
 (u"McDonald's", 26),
 (u"Wendy's", 12),
 (u'Burger King', 11),
 (u'Chick-fil-A', 5)]


## Additional exploration

### top 5 common ammenities

In [70]:
cur.execute ("SELECT value, COUNT(*) as num \
FROM nodes_tags \
WHERE key='amenity'\
GROUP BY value \
ORDER BY num DESC \
LIMIT 10;")

pprint.pprint(cur.fetchall())

[(u'restaurant', 470),
 (u'place_of_worship', 357),
 (u'bicycle_parking', 263),
 (u'school', 233),
 (u'fast_food', 227),
 (u'fuel', 147),
 (u'fountain', 101),
 (u'bench', 99),
 (u'bank', 83),
 (u'cafe', 54)]


### top 5 religions

In [72]:
cur.execute("SELECT nodes_tags.value, COUNT(*) as num \
FROM nodes_tags \
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i \
    ON nodes_tags.id=i.id \
WHERE nodes_tags.key='religion' \
GROUP BY nodes_tags.value \
ORDER BY num DESC \
LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u'christian', 335),
 (u'buddhist', 2),
 (u'jewish', 2),
 (u'scientologist', 2),
 (u'bahai', 1)]


## Conclusion and further improvement suggestions

### Other ideas about the datasets

OpenStreetMap for city of tampa has reasonable quality, it provides limited but quite reliable information, as we saw some expected results in data exploration section. I have used basic data auditing and data cleaning techniques to programatically fixed large amount of data, significant problems are still exists:
* lack of user participation and timely update. To this point, gamification can be implemented to encourage user participation through incentives, perhaps binding the map with some feature like pokemon go would help.
* Automated data auditing may be implemented through some Cross-referencing/Cross-validating by plugging in Google API.

173 users input data only once. above mentioned gamification has the potential to engage more users and encourage participation.

In [36]:
cur.execute("SELECT COUNT(*) FROM \
    (SELECT e.user, COUNT(*) as num \
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
     GROUP BY e.user \
     HAVING num=1)  u;")

pprint.pprint(cur.fetchall())

[(173,)]


### Benefits and Anticipated Problems in Implementing the Improvement

* Benefits: More data input, more potential to extend OpenStreetMap. This open source allows people to include travel, walking and biking tour information, user reviews of establishments, local public transit etc. This could expand OpenStreetMap's user base, and become more consumer focused. Also, increased utility of this map will attract more local businesses to take care of data quality related to them.
* Anticipated Problems: OpenStreeMap has no paid employees, the growth and expansion only based on volunteer users, additionaly, there is learning curve to figure out what OpenStreeMap is about(myself as an example), therefore it's hard to attract users especially some easy-to-use map apps like Google map are so dominant.

-------------------------end of report-------------------------


--------------------------


### working code

In [1]:
import os
import os.path
import sys
import time
import requests
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
from collections import defaultdict


In [2]:
osm_sample = 'temple_terrace.osm'
osm_filename = 'tampa.osm'
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

tags = count_tags(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in tags.items()], reverse=True)]

print 'Element types and occurrence of tampa.osm'
pprint.pprint(sorted_by_occurrence)


Element types and occurrence of tampa.osm
[('nd', 2195579),
 ('node', 1822389),
 ('tag', 916600),
 ('way', 291596),
 ('member', 17721),
 ('relation', 670),
 ('osm', 1),
 ('bounds', 1)]


In [3]:
##regular expressions to define tag key characters
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(element.attrib['k']):
                keys['lower'] = keys['lower'] + 1
            elif lower_colon.search(element.attrib['k']):
                keys['lower_colon'] = keys['lower_colon'] + 1
            elif problemchars.search(element.attrib['k']):
                print 'problemchars:',k
                keys['problemchars'] = keys['problemchars'] + 1
            else:
                #print k
                keys['other'] = keys['other'] + 1
    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

print 'Key types and occurrence of tampa.osm'
pprint.pprint(process_map(osm_filename))

Key types and occurrence of tampa.osm
problemchars: service area
problemchars: Payments Accepted
problemchars: Payments Accepted
{'lower': 710176, 'lower_colon': 194348, 'other': 12073, 'problemchars': 3}


In [4]:
def count_keys(filename):
    keys = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key:
                if key not in keys:
                    keys[key] = 1
                else:
                    keys[key] += 1
    return keys

keys = count_keys(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in keys.items()], reverse=True)]

print 'Keys and occurrence in tampa.osm'
pprint.pprint(sorted_by_occurrence[0:20]) ##print 20 most frequent keys


Keys and occurrence in tampa.osm
[('building', 225093),
 ('height', 216193),
 ('highway', 62865),
 ('name', 38998),
 ('tiger:county', 23293),
 ('tiger:cfcc', 23257),
 ('tiger:name_base', 21924),
 ('tiger:name_type', 21637),
 ('tiger:reviewed', 20105),
 ('tiger:zip_left', 18477),
 ('tiger:zip_right', 17650),
 ('source', 12530),
 ('oneway', 11169),
 ('power', 9668),
 ('surface', 7776),
 ('lanes', 7449),
 ('maxspeed', 6108),
 ('natural', 5882),
 ('service', 5632),
 ('amenity', 5630)]


In [5]:
def count_postcodes(filename):
    postcodes = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key == 'addr:postcode':
                postcode = elem.attrib.get('v')
                if postcode not in postcodes:
                    postcodes[postcode] = 1
                else:
                    postcodes[postcode] += 1
    return postcodes


#start_time = time.time()

postcodes = count_postcodes(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in postcodes.items()], reverse=True)]

print 'Postcode values and occurrence in tampa.osm'
pprint.pprint(sorted_by_occurrence)

#print('\n--- %s seconds ---' % (time.time() - start_time))

Postcode values and occurrence in tampa.osm
[('33612', 167),
 ('33702', 120),
 ('33511', 109),
 ('33607', 92),
 ('33602', 89),
 ('33703', 88),
 ('33704', 79),
 ('33781', 74),
 ('33606', 71),
 ('33714', 67),
 ('33617', 67),
 ('33624', 64),
 ('33716', 59),
 ('33635', 59),
 ('33578', 57),
 ('33626', 49),
 ('34677', 45),
 ('33614', 41),
 ('33619', 37),
 ('33634', 34),
 ('33609', 32),
 ('33569', 30),
 ('33527', 27),
 ('33604', 26),
 ('33605', 24),
 ('33629', 23),
 ('33603', 22),
 ('33615', 21),
 ('33611', 21),
 ('33610', 21),
 ('33556', 20),
 ('33613', 19),
 ('33713', 18),
 ('33594', 18),
 ('33592', 18),
 ('33618', 16),
 ('33762', 14),
 ('33547', 14),
 ('33729', 13),
 ('33710', 12),
 ('33596', 8),
 ('33647', 7),
 ('33544', 7),
 ('33572', 6),
 ('33558', 6),
 ('33534', 6),
 ('34655', 5),
 ('33637', 5),
 ('33584', 5),
 ('33559', 5),
 ('34639', 4),
 ('33549', 4),
 ('33625', 3),
 ('33620', 3),
 ('33616', 3),
 ('34695', 2),
 ('34677-6300', 2),
 ('33643', 2),
 ('33579', 2),
 ('33548', 2),
 ('33510

In [18]:

def clean_postcode(postcode_value):

    if len(postcode_value)!=5:
        postcode=postcode_value[0:5]
                        
        return postcode
    #return tree.write(cleaned_filename)


#start_time = time.time()

#cleaned_postcode = 'tampa_cleaned_postcode.xml'
#clean_postcode(osm_filename, cleaned_postcode)


#print('\n--- %s seconds ---' % (time.time() - start_time))

In [7]:
#postcodes = count_postcodes('tampa_cleaned_postcode.xml')
#sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in postcodes.items()], reverse=True)]

#print 'Postcode values and occurrence after cleaning:\n'
#pprint.pprint(sorted_by_occurrence)

In [8]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "av":"Avenue",
            "ave": "Avenue",
           'Ave.':"Avenue",
           'Blvd':"Boulevard",
           'Blvd.':"Boulevard",
           'Bolevard':"Boulevard",
           'Boulvard':"Boulevard",
           'Cir':"Circle",
           'Cswy':"Causeway",
           'Ct':"Court",
           'Dr':"Drive",
           'Dr.':"Drive",
           'Hwy':"Highway",
           'Ln':"Lane",
           'Pkwy':"Parkway",
           'Pky':"Parking",
           'Pl':"Plaza",
           'Rd':"Road",
           'Rd.':"Road",
           'St':"Street",
           'street':"Street",
           'dr':"Drive",
           'drive':"Drive"          
}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:street":
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

audit(osm_filename)


defaultdict(set,
            {'1002': {'N. Westshore BLVD #1002'},
             '101': {'Ulmerton Rd, Suite 101'},
             '102': {'FL-54 #102'},
             '103': {'Ulmerton Rd, Suite 103'},
             '105': {'S Howard Av 105'},
             '106': {'Ulmerton Rd, Suite 106'},
             '107': {'Ulmerton Rd, Suite 107'},
             '131': {'E Fletcher Ave #131'},
             '19': {'US 19'},
             '201': {'Ulmerton Rd, Suite 201'},
             '202': {'Ulmerton Rd, Suite 202'},
             '203': {'Ulmerton Rd, Suite 203'},
             '206': {'Ulmerton Rd, Suite 206'},
             '250': {'North Armenia Avenue Suite #250'},
             '300': {'W Palmira Ave #300'},
             '301': {'S US Highway 301',
              'South US Highway 301',
              'US 301',
              'US Highway 301',
              'us Highway 301'},
             '33635': {'8492 Manatee Bay Dr Tampa, FL 33635'},
             '41': {'N US Highway 41'},
             '54': {'FL 5

In [10]:
def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type_re, mapping[street_type], name)

    return name

street_types = audit(osm_filename)

for st_type, ways in street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        if name != better_name:
            print name, "=>", better_name



Penrod Ln => Penrod Lane
Whitmarsh Ln => Whitmarsh Lane
Webb Rd => Webb Road
Hutchison Rd => Hutchison Road
Race Track Rd => Race Track Road
Kelly Rd => Kelly Road
Providence Rd => Providence Road
Anderson Rd => Anderson Road
Orient Rd => Orient Road
Tampa Rd => Tampa Road
Ulmerton Rd => Ulmerton Road
George Rd => George Road
Tarpon Springs Rd => Tarpon Springs Road
Ehrlich Rd => Ehrlich Road
Sheldon Rd => Sheldon Road
Palm Pointe Dr. => Palm Pointe Drive
9201 W. Waters Ave. => 9201 W. Waters Avenue
E. Fowler Ave. => E. Fowler Avenue
8203 Peterson Rd. => 8203 Peterson Road
W Lumsden Rd. => W Lumsden Road
University Pl => University Plaza
17501 N Palms Village Pl => 17501 N Palms Village Plaza
Harvest Pl => Harvest Plaza
benjamin center dr => benjamin center Drive
Markstown drive => Markstown Drive
N. Dale Mabry Hwy => N. Dale Mabry Highway
Bullard Pkwy => Bullard Parkway
Carillon Pkwy => Carillon Parkway
Harbour Post Dr => Harbour Post Drive
Citrus Park Dr => Citrus Park Drive
Rocky Cr

In [11]:
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")


def clean_street_name(filename):
    tree = ET.parse(filename)
    root = tree.getroot()

    for tag in root.findall('*/tag'):
        if is_street_name(tag):
            name = tag.get('v')
            better_name = update_name(name, mapping)
            tag.set('v', better_name)

    #return tree.write(cleaned_filename)

#cleaned_street_name = 'tampa_cleaned.xml'
#clean_street_name(cleaned_postcode, cleaned_street_name)


In [12]:
# %load 'schema.py'

# Note: The schema is stored in a .py file in order to take advantage of the
# int() and float() type coercion functions. Otherwise it could easily stored as
# as JSON or another serialized format.

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [26]:
import csv

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""





# Make sure the fields order in the csvs matches the column order in the
# sql table schema
OSM_PATH = 'temple_terrace.osm'
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {} 
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):  
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                node_tag = {} 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = tag.attrib['v']  

                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    node_tag['type'] = 'regular'
                    node_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+):', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])
                    node_tag['type'] = bef_colon[0]
                    node_tag['key'] = aft_colon[0]
                    if node_tag['type'] == "addr" and node_tag['key'] == "street":
                        # update street name
                        node_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif node_tag['type'] == "addr" and node_tag['key'] == "postcode":
                        # update post code
                        node_tag['value'] = clean_postcode(tag.attrib['v']) 


            tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                way_tag = {}
                way_tag['id'] = element.attrib['id'] 
                way_tag['value'] = tag.attrib['v']
                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+?):+[a-z]', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])

                    way_tag['type'] = bef_colon[0]
                    way_tag['key'] = aft_colon[0]
                    if way_tag['type'] == "addr" and way_tag['key'] == "street":
                        way_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif way_tag['type'] == "addr" and way_tag['key'] == "postcode":
                        way_tag['value'] = clean_postcode(tag.attrib['v']) 
            tags.append(way_tag)
        position = 0
        for tag in element.iter("nd"):  
            nd = {}
            nd['id'] = element.attrib['id'] 
            nd['node_id'] = tag.attrib['ref'] 
            nd['position'] = position  
            position += 1
            
            way_nodes.append(nd)
    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()



        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)


In [20]:
import os
print('The tampa_cleaned.xml file is {} MB'.format(os.path.getsize('tampa.osm')/1.0e6))
print('The tampa.db file is {} MB'.format(os.path.getsize('tampa.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

The tampa_cleaned.xml file is 417.46995 MB
The tampa.db file is 147.521536 MB
The nodes.csv file is 161.004855 MB
The nodes_tags.csv file is 161.004855 MB
The ways.csv file is 19.066869 MB
The ways_tags.csv is 27.623908 MB
The ways_nodes.csv is 52.368008 MB


In [28]:
import sqlite3
sqlite_file = "tampa.db"
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

In [29]:
cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())

[(1048575,)]


In [51]:
cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(291596,)]


In [30]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print(cur.fetchall())

[(893,)]


In [53]:
import pprint
cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())

[(u'Andrew Matheny_import', 404260),
 (u'ninja_import', 94536),
 (u'EdHillsman', 91743),
 (u'woodpeck_fixbot', 59404),
 (u'coleman', 58410),
 (u'David Hey', 57929),
 (u'KalininOV', 57366),
 (u'grouper', 39270),
 (u'jharpster', 37479),
 (u'Jack the Ripper', 32749)]


In [54]:
cur.execute("SELECT COUNT(*) FROM nodes_tags WHERE value LIKE '%Starbucks%';")
print(cur.fetchall())

[(28,)]


In [55]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%tourism'\
             GROUP BY tags.value \
             ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[(u'hotel', 115),
 (u'information', 57),
 (u'motel', 50),
 (u'attraction', 21),
 (u'viewpoint', 16),
 (u'museum', 15),
 (u'artwork', 12),
 (u'picnic_site', 12),
 (u'theme_park', 9),
 (u'camp_site', 6),
 (u'zoo', 4),
 (u'caravan_site', 3),
 (u'terminal', 2),
 (u'apartment', 1),
 (u'aquarium', 1),
 (u'guest_house', 1),
 (u'hostel', 1),
 (u'zoo;attraction', 1)]


In [56]:

import pprint
cur.execute("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
            FROM nodes_tags WHERE value = 'restaurant') i ON nodes_tags.id = i.id WHERE nodes_tags.key = 'city'\
            GROUP BY nodes_tags.value ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[(u'Tampa', 129),
 (u'St. Petersburg', 24),
 (u'Saint Petersburg', 17),
 (u'Brandon', 14),
 (u'Clearwater', 5),
 (u'Oldsmar', 5),
 (u'Pinellas Park', 4),
 (u'Riverview', 3),
 (u'Apollo Beach', 1),
 (u'Dover', 1),
 (u'Gibsonton', 1),
 (u'Temple Terrace', 1),
 (u'Valrico', 1)]


In [57]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='cafe') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u'Starbucks', 15),
 (u"Dunkin' Donuts", 3),
 (u'Starbucks Coffee', 3),
 (u'Kaleisia Tea Lounge', 2),
 (u'Tropical Smoothie Cafe', 2)]
